In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import random
import seaborn as sn

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1, l2, l1_l2
from sklearn.preprocessing import minmax_scale, OneHotEncoder
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dropout

In [ ]:
home_dir = "C:/Users/ernest.liu/Documents/ZZZ Misc/UMAFall/UMAFall_Dataset Formatted/"

## Autoencoder approach

In [ ]:
os.chdir(home_dir)
all_files = os.listdir()
adl_files = []
fall_files = []
for file in all_files:
    if "adl" in file.lower():
        adl_files.append(file)
    else:
        fall_files.append(file)

### Data Preprocessing

In [ ]:
x_axis, y_axis, z_axis = [], [], []
for file in adl_files:
    data = pd.read_csv(file)
    sensor0 = data[(data[" Sensor Type"] == 0) & (data[" Sensor ID"] == 0)].iloc[:, range(6)]
    sensor0.iloc[:,2] = sensor0.iloc[:,2].astype(np.float32)
    sensor0.iloc[:,3] = sensor0.iloc[:,3].astype(np.float32)
    sensor0.iloc[:,4] = sensor0.iloc[:,4].astype(np.float32)
    sensor0 = sensor0.groupby("% TimeStamp", as_index = False).mean()
    x = sensor0[' X-Axis']
    y = sensor0[' Y-Axis']
    z = sensor0[' Z-Axis']
    x_axis.append(x)
    y_axis.append(y)
    z_axis.append(z)
    print(f"{file} completed")

In [ ]:
x_axis_fall, y_axis_fall, z_axis_fall = [], [], []
for file in fall_files:
    data = pd.read_csv(file)
    sensor0 = data[(data[" Sensor Type"] == 0) & (data[" Sensor ID"] == 0)].iloc[:, range(6)]
    sensor0.iloc[:,2] = sensor0.iloc[:,2].astype(np.float32)
    sensor0.iloc[:,3] = sensor0.iloc[:,3].astype(np.float32)
    sensor0.iloc[:,4] = sensor0.iloc[:,4].astype(np.float32)
    sensor0 = sensor0.groupby("% TimeStamp", as_index = False).mean()
    x = sensor0[' X-Axis']
    y = sensor0[' Y-Axis']
    z = sensor0[' Z-Axis']
    x_axis_fall.append(x)
    y_axis_fall.append(y)
    z_axis_fall.append(z)
    print(f"{file} completed")

In [ ]:
x_min_count = 4000
for d in x_axis:
    if x_min_count > len(d):
        x_min_count = len(d)
for d in x_axis_fall:
    if x_min_count > len(d):
        x_min_count = len(d)
for i, arr in enumerate(x_axis):
    x_axis[i] = arr[:x_min_count]
for i, arr in enumerate(x_axis_fall):
    x_axis_fall[i] = arr[:x_min_count]
x_axis = np.array(x_axis)
x_axis_fall = np.array(x_axis_fall)
        
y_min_count = 4000
for d in y_axis:
    if y_min_count > len(d):
        y_min_count = len(d)
for d in y_axis_fall:
    if y_min_count > len(d):
        y_min_count = len(d)
for i, arr in enumerate(y_axis):
    y_axis[i] = arr[:y_min_count]
for i, arr in enumerate(y_axis_fall):
    y_axis_fall[i] = arr[:y_min_count]
y_axis = np.array(y_axis)
y_axis_fall = np.array(y_axis_fall)
        
z_min_count = 4000
for d in z_axis:
    if z_min_count > len(d):
        z_min_count = len(d)
for d in z_axis_fall:
    if z_min_count > len(d):
        z_min_count = len(d)
for i, arr in enumerate(z_axis):
    z_axis[i] = arr[:z_min_count]
for i, arr in enumerate(z_axis_fall):
    z_axis_fall[i] = arr[:z_min_count]
z_axis = np.array(z_axis)
z_axis_fall = np.array(z_axis_fall)

In [ ]:
x_axis.shape

In [ ]:
x_train_data_clean = np.delete(x_axis, [304,305,306], axis = 0)
x_train_data_clean = preprocessing.normalize(x_train_data_clean)
x_axis_fall_clean = preprocessing.normalize(x_axis_fall)

In [ ]:
y_axis_clean = preprocessing.normalize(y_axis)
y_axis_fall_clean = preprocessing.normalize(y_axis_fall)

In [ ]:
y_jumps = sorted(abs((y_axis_clean[:,:-1] - y_axis_clean[:,1:]).flatten()), reverse=True)

In [ ]:
print(np.unique(np.where(abs(y_axis_clean[:,:-1] - y_axis_clean[:,1:]) >= 0.2)[0]))
print("Therefore, we remove rows 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 from our dataset")

In [ ]:
y_axis_clean = np.delete(y_axis_clean, range(299,316), axis = 0)

In [ ]:
np.max(abs(y_axis_clean[:,:-1] - y_axis_clean[:,1:]))

In [ ]:
index = 335
plt.grid()
plt.plot(np.arange(len(y_axis_clean[index])), y_axis_clean[index])
plt.title("ADL y-axis")
plt.show()

In [ ]:
z_axis_clean = preprocessing.normalize(z_axis)
z_axis_fall_clean = preprocessing.normalize(z_axis_fall)

In [ ]:
z_jumps = sorted(abs((z_axis_clean[:,:-1] - z_axis_clean[:,1:]).flatten()), reverse=True)

In [ ]:
to_remove_z = np.unique(np.where(abs(z_axis_clean[:,:-1] - z_axis_clean[:,1:]) >= 0.2)[0])
print(to_remove_z)
print(f"Therefore, we remove rows {to_remove_z} from our dataset")

In [ ]:
z_axis_clean = np.delete(z_axis_clean, to_remove_z, axis = 0)

In [ ]:
index = 2
plt.grid()
plt.plot(np.arange(len(z_axis_clean[index])), z_axis_clean[index])
plt.title("ADL z-axis")
plt.show()

In [ ]:
ratio = 0.8

x_train_data = x_train_data_clean[:round(len(x_train_data_clean)*ratio)]
x_test_data = x_train_data_clean[round(len(x_train_data_clean)*ratio):]


y_train_data = y_axis_clean[:round(len(y_axis_clean)*ratio)]
y_test_data = y_axis_clean[round(len(y_axis_clean)*ratio):]

# Not done
z_train_data = z_axis_clean[:round(len(z_axis_clean)*ratio)]
z_test_data = z_axis_clean[round(len(z_axis_clean)*ratio):]

In [ ]:
index = 306
plt.grid()
plt.plot(np.arange(len(x_train_data[index])), x_train_data[index])
plt.title("ADL x-axis")
plt.show()

### Building Autoencoder

In [ ]:
class Autoencoder(Model):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            layers.Dense(500, activation = 'relu')
            #layers.Dense(1000, activation = 'relu'),
            #layers.Dense(900, activation = 'relu'),
            #layers.Dense(800, activation = 'relu')
        ])
        
        self.decoder = tf.keras.Sequential([
            #layers.Dense(900, activation = 'relu'),
            #layers.Dense(1000, activation = 'relu'),
            layers.Dense(1173, activation = 'sigmoid')
        ])
        
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
autoencoder_x = Autoencoder()
autoencoder_y = Autoencoder()
autoencoder_z = Autoencoder()

In [ ]:
autoencoder_x.compile(optimizer = 'adam', loss = 'mae')
autoencoder_y.compile(optimizer = 'adam', loss = 'mae')
autoencoder_z.compile(optimizer = 'adam', loss = 'mae')

early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
history_x = autoencoder_x.fit(x_train_data, x_train_data,
                         epochs = 100000,
                         batch_size = 32,
                         validation_data = (x_test_data, x_test_data),
                         shuffle = True, callbacks = [early_stopping])

In [ ]:
history_y = autoencoder_y.fit(y_train_data, y_train_data,
                         epochs = 100000,
                         batch_size = 32,
                         validation_data = (y_test_data, y_test_data),
                         shuffle = True, callbacks = [early_stopping])

In [ ]:
history_z = autoencoder_z.fit(z_train_data, z_train_data,
                         epochs = 100000,
                         batch_size = 32,
                         validation_data = (z_test_data, z_test_data),
                         shuffle = True, callbacks = [early_stopping])

### Inspecting results

In [ ]:
def mae(y_true, predictions):
    y_true, predictions = np.array(y_true), np.array(predictions)
    return np.mean(np.abs(y_true - predictions))

In [ ]:
predicted_x = autoencoder_x.predict(x_train_data_clean)

In [ ]:
predicted_x.shape

In [ ]:
index = 0

In [ ]:
print(index)
print(f"MAE: {mae(predicted_x[index], x_train_data_clean[index])}")
plt.plot(predicted_x[index], 'r')
plt.plot(x_train_data_clean[index], 'b')

plt.fill_between(np.arange(1173), predicted_x[index], x_train_data_clean[index], color = 'lightcoral')
plt.legend(labels = ['Reconstruction', 'Input', 'Error'])
plt.show()
index += 1

In [ ]:
predicted_y = autoencoder_y.predict(y_train_data)

In [ ]:
index = 0

In [ ]:
print(index)
print(f"MAE: {mae(predicted_y[index], y_train_data[index])}")
plt.plot(predicted_y[index], 'r')
plt.plot(y_train_data[index], 'b')

plt.fill_between(np.arange(1173), predicted_y[index], y_train_data[index], color = 'lightcoral')
plt.legend(labels = ['Reconstruction', 'Input', 'Error'])
plt.show()
index += 1

In [ ]:
mae_scores = [mae(predicted_y[i], y_train_data[i]) for i in range(y_train_data.shape[0])]

In [ ]:
sorted(mae_scores)

### Determining Threshold

In [ ]:
adl_loss_x = tf.keras.losses.mae(predicted_x, x_train_data_clean)
plt.hist(adl_loss_x[None,:], bins = 50)
plt.xlabel("ADL loss")
plt.ylabel("No of examples")
plt.show()

In [ ]:
predicted_fall = autoencoder_x.predict(x_axis_fall_clean)

In [ ]:
fall_loss_x = tf.keras.losses.mae(predicted_fall, x_axis_fall_clean)
plt.hist(fall_loss_x[None,:], bins = 50)
plt.xlabel("Fall loss")
plt.ylabel("No of examples")
plt.show()

In [ ]:
threshold = np.mean(fall_loss_x) - np.std(fall_loss_x)
print(f"threshold: {threshold}")

In [ ]:
def compute_evaluation(adl_loss, fall_loss, threshold):
    train_total = len(adl_loss)
    test_total = len(fall_loss)

    tp = np.count_nonzero(fall_loss > threshold) # right side of fall
    fp = np.count_nonzero(adl_loss > threshold) # right side of adl
    fn = np.count_nonzero(fall_loss <= threshold) # left side of fall
    tn = np.count_nonzero(adl_loss <= threshold) # left side of adl
    print(f'True Positives: {tp}')
    print(f'False Positives: {fp}')
    print(f'False Negatives: {fn}')
    print(f'True Negatives: {tn}')
    print()

    acc = accuracy(adl_loss, fall_loss, threshold)
    prec = precision(adl_loss, fall_loss, threshold)
    rec = recall(adl_loss, fall_loss, threshold)
    f1_score = f1(adl_loss, fall_loss, threshold)
    f1_weighted = weighted_f1(adl_loss, fall_loss, threshold) # prioritising precision
    print('------------------')
    print('Evalutaion Metrics')
    print('------------------')
    print(f'Accuracy: {round(acc,3)}')
    print(f'Precision: {round(prec,3)}')
    print(f'Recall: {round(rec,3)}')
    print(f'F1 Score: {round(f1_score,3)}')
    print(f'Weighted F1 Score: {round(f1_weighted,3)}')


def accuracy(adl_loss, fall_loss, threshold):
    tp = np.count_nonzero(fall_loss > threshold) # right side of fall
    fp = np.count_nonzero(adl_loss > threshold) # right side of adl
    fn = np.count_nonzero(fall_loss <= threshold) # left side of fall
    tn = np.count_nonzero(adl_loss <= threshold) # left side of adl
    return (tp+tn)/(tp+fp+fn+tn)

def precision(adl_loss, fall_loss, threshold):
    tp = np.count_nonzero(fall_loss > threshold) # right side of fall
    fp = np.count_nonzero(adl_loss > threshold) # right side of adl
    fn = np.count_nonzero(fall_loss <= threshold) # left side of fall
    tn = np.count_nonzero(adl_loss <= threshold) # left side of adl
    try:
        return tp/(tp+fp)
    except: return 0

def recall(adl_loss, fall_loss, threshold):
    tp = np.count_nonzero(fall_loss > threshold) # right side of fall
    fp = np.count_nonzero(adl_loss > threshold) # right side of adl
    fn = np.count_nonzero(fall_loss <= threshold) # left side of fall
    tn = np.count_nonzero(adl_loss <= threshold) # left side of adl
    try:
        return tp/(tp+fn)
    except: return 0

def f1(adl_loss, fall_loss, threshold):
    tp = np.count_nonzero(fall_loss > threshold) # right side of fall
    fp = np.count_nonzero(adl_loss > threshold) # right side of adl
    fn = np.count_nonzero(fall_loss <= threshold) # left side of fall
    tn = np.count_nonzero(adl_loss <= threshold) # left side of adl
    prec = precision(adl_loss, fall_loss, threshold)
    rec = recall(adl_loss, fall_loss, threshold)
    try:
        return 2*((prec*rec)/(prec+rec))
    except: return 0

def weighted_f1(adl_loss, fall_loss, threshold, beta=0.7):

    tp = np.count_nonzero(fall_loss > threshold) # right side of fall
    fp = np.count_nonzero(adl_loss > threshold) # right side of adl
    fn = np.count_nonzero(fall_loss <= threshold) # left side of fall
    tn = np.count_nonzero(adl_loss <= threshold) # left side of adl

    prec = precision(adl_loss, fall_loss, threshold)
    rec = recall(adl_loss, fall_loss, threshold)
    try:
        return (1+beta**2) * ((prec*rec)/(beta**2*prec+rec))
    except:
        return 0


    
compute_evaluation(adl_loss_x, fall_loss_x, threshold)

In [ ]:
min_threshold = min(np.min(adl_loss_x), np.min(fall_loss_x))
max_threshold = max(np.max(adl_loss_x), np.max(fall_loss_x))
threshold_list = np.linspace(min_threshold, max_threshold, num = 100)

In [ ]:
weighted_f1_score_list = [weighted_f1(adl_loss_x, fall_loss_x, threshold, beta = 0.5) for threshold in threshold_list]
precision_list = [precision(adl_loss_x, fall_loss_x, threshold) for threshold in threshold_list]
recall_list = [recall(adl_loss_x, fall_loss_x, threshold) for threshold in threshold_list]

In [ ]:
plt.plot(threshold_list, weighted_f1_score_list)
plt.xlabel("Threshold")
plt.ylabel("weighted f1 score list")
plt.show()

In [ ]:
index = np.argmax(weighted_f1_score_list)
print(f"Highest Weighted F1 Score: {weighted_f1_score_list[index]}")
print(f"Best Threshold: {threshold_list[index]}")

In [ ]:
accuracy(adl_loss_x, fall_loss_x, threshold_list[index])

In [ ]:
precision(adl_loss_x, fall_loss_x, threshold_list[index])

In [ ]:
recall(adl_loss_x, fall_loss_x, threshold_list[index])

In [ ]:
adl_loss_y = tf.keras.losses.mae(predicted_y, y_train_data)
plt.hist(adl_loss_y[None,:], bins = 50)
plt.xlabel("ADL loss")
plt.ylabel("No of examples")
plt.show()

In [ ]:
predicted_fall = autoencoder_y.predict(y_axis_fall_clean)

In [ ]:
fall_loss_y = tf.keras.losses.mae(predicted_fall, y_axis_fall_clean)
plt.hist(fall_loss_y[None,:], bins = 50)
plt.xlabel("Fall loss")
plt.ylabel("No of examples")
plt.show()

In [ ]:
threshold = np.mean(fall_loss_y) - np.std(fall_loss_y)
print(f"threshold: {threshold}")

In [ ]:
min_threshold = min(np.min(adl_loss_y), np.min(fall_loss_y))
max_threshold = max(np.max(adl_loss_y), np.max(fall_loss_y))
threshold_list = np.linspace(min_threshold, max_threshold, num = 100)

In [ ]:
weighted_f1_score_list = [weighted_f1(adl_loss_y, fall_loss_y, threshold, beta = 0.5) for threshold in threshold_list]
precision_list = [precision(adl_loss_y, fall_loss_y, threshold) for threshold in threshold_list]
recall_list = [recall(adl_loss_y, fall_loss_y, threshold) for threshold in threshold_list]

In [ ]:
plt.plot(threshold_list, weighted_f1_score_list)
plt.xlabel("Threshold")
plt.ylabel("weighted f1 score list")
plt.show()

In [ ]:
index = np.argmax(weighted_f1_score_list)
print(f"Highest Weighted F1 Score: {weighted_f1_score_list[index]}")
print(f"Best Threshold: {threshold_list[index]}")

# ANN Approach

In [ ]:
x_adl = x_train_data_clean
x_fall = x_axis_fall_clean

y_adl = y_axis_clean
y_fall = y_axis_fall_clean

z_adl = z_axis_clean
z_fall = z_axis_fall_clean

In [ ]:
x_data = np.concatenate([x_adl, x_fall])
y_data = np.concatenate([y_adl, y_fall])
z_data = np.concatenate([z_adl, z_fall])

print(f"""
x_data: {x_data.shape[0]}
y_data: {y_data.shape[0]}
z_data: {z_data.shape[0]}
""")

In [ ]:
x_labels = np.concatenate([np.zeros((x_adl.shape[0], 1)), np.ones((x_fall.shape[0], 1))], axis = 0)
y_labels = np.concatenate([np.zeros((y_adl.shape[0], 1)), np.ones((y_fall.shape[0], 1))], axis = 0)
z_labels = np.concatenate([np.zeros((z_adl.shape[0], 1)), np.ones((z_fall.shape[0], 1))], axis = 0)

In [ ]:
test_percent = 0.2
np.random.seed(100)
x_test_ind = sorted(np.random.choice(range(x_data.shape[0]), replace = False, size = round(x_data.shape[0] * test_percent)))
y_test_ind = sorted(np.random.choice(range(y_data.shape[0]), replace = False, size = round(y_data.shape[0] * test_percent)))
z_test_ind = sorted(np.random.choice(range(z_data.shape[0]), replace = False, size = round(z_data.shape[0] * test_percent)))
x_train_ind = np.delete(np.arange(0, x_data.shape[0]), x_test_ind)
y_train_ind = np.delete(np.arange(0, y_data.shape[0]), y_test_ind)
z_train_ind = np.delete(np.arange(0, z_data.shape[0]), z_test_ind)

In [ ]:
x_train_data = x_data[x_train_ind, :]
y_train_data = y_data[y_train_ind, :]
z_train_data = z_data[z_train_ind, :]

x_train_labels = x_labels[x_train_ind, :]
y_train_labels = y_labels[y_train_ind, :]
z_train_labels = z_labels[z_train_ind, :]

x_test_data = x_data[x_test_ind, :]
y_test_data = y_data[y_test_ind, :]
z_test_data = z_data[z_test_ind, :]

x_test_labels = x_labels[x_test_ind, :]
y_test_labels = y_labels[y_test_ind, :]
z_test_labels = z_labels[z_test_ind, :]

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(x_train_labels)

x_train_labels = enc.transform(x_train_labels).toarray()
x_test_labels = enc.transform(x_test_labels).toarray()

y_train_labels = enc.transform(y_train_labels).toarray()
y_test_labels = enc.transform(y_test_labels).toarray()

z_train_labels = enc.transform(z_train_labels).toarray()
z_test_labels = enc.transform(z_test_labels).toarray()

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)

In [ ]:
model_x = tf.keras.Sequential([
    layers.Dense(400, activation = 'relu'),
    layers.Dense(300, activation = 'relu'),
    layers.Dense(150, activation = 'relu'),
    layers.Dense(2, activation = 'sigmoid')
])

model_y = tf.keras.Sequential([
    layers.Dense(400, activation = 'relu'),
    layers.Dense(300, activation = 'relu'),
    layers.Dense(150, activation = 'relu'),
    layers.Dense(2, activation = 'sigmoid')
])

model_z = tf.keras.Sequential([
    layers.Dense(400, activation = 'relu'),
    layers.Dense(300, activation = 'relu'),
    layers.Dense(150, activation = 'relu'),
    layers.Dense(2, activation = 'sigmoid')
])

model_x.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model_y.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model_z.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
history_x = model_x.fit(x_train_data, x_train_labels,
                    validation_data = (x_test_data, x_test_labels),
                    epochs = 100,
                    callbacks = [early_stopping],
                    shuffle = True)

In [ ]:
history_y = model_y.fit(y_train_data, y_train_labels,
                    validation_data = (y_test_data, y_test_labels),
                    epochs = 100,
                    callbacks = [early_stopping],
                    shuffle = True)

In [ ]:
history_z = model_z.fit(z_train_data, z_train_labels,
                    validation_data = (z_test_data, z_test_labels),
                    epochs = 100,
                    callbacks = [early_stopping],
                    shuffle = True)

In [ ]:
predicted_x = model_x.predict(x_test_data)
predicted_x = [np.argmax(i) for i in predicted_x]
actual_x = [np.argmax(i) for i in x_test_labels]

cf_matrix = confusion_matrix(predicted_x, actual_x)

ax = sn.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')
ax.set_title('Seaborn Confusion Matrix with labels\n\n')
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values')
## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['ADL','Fall'])
ax.yaxis.set_ticklabels(['ADL','Fall'])
## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
predicted_y = model_y.predict(y_test_data)
predicted_y = [np.argmax(i) for i in predicted_y]
actual_y = [np.argmax(i) for i in y_test_labels]

cf_matrix = confusion_matrix(predicted_y, actual_y)

ax = sn.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')
ax.set_title('Seaborn Confusion Matrix with labels\n\n')
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values')
## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['ADL','Fall'])
ax.yaxis.set_ticklabels(['ADL','Fall'])
## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
predicted_z = model_z.predict(z_test_data)
predicted_z = [np.argmax(i) for i in predicted_z]
actual_z = [np.argmax(i) for i in z_test_labels]

cf_matrix = confusion_matrix(predicted_z, actual_z)

ax = sn.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')
ax.set_title('Seaborn Confusion Matrix with labels\n\n')
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values')
## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['ADL','Fall'])
ax.yaxis.set_ticklabels(['ADL','Fall'])
## Display the visualization of the Confusion Matrix.
plt.show()

# 1D CNN Approach

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3)

In [ ]:
cnn_model_x = tf.keras.Sequential([
    Conv1D(filters = 64, kernel_size = 3, activation = 'relu'),
    Conv1D(filters = 64, kernel_size = 3, activation = 'relu'),
    Dropout(0.5),
    MaxPooling1D(pool_size = 2),
    Flatten(),
    layers.Dense(400, activation = 'relu'),
    layers.Dense(300, activation = 'relu'),
    layers.Dense(150, activation = 'relu'),
    layers.Dense(2, activation = 'sigmoid')
])

cnn_model_x.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
x_train_data_reshaped = x_train_data[..., None]
x_test_data_reshaped = x_test_data[..., None]

In [ ]:
history_x = cnn_model_x.fit(x_train_data_reshaped, x_train_labels,
                    validation_data = (x_test_data_reshaped, x_test_labels),
                    epochs = 100,
                    callbacks = [early_stopping],
                    shuffle = True)

In [ ]:
predicted_x = cnn_model_x.predict(x_test_data)
predicted_x = [np.argmax(i) for i in predicted_x]
actual_x = [np.argmax(i) for i in x_test_labels]

cf_matrix = confusion_matrix(predicted_x, actual_x)

ax = sn.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')
ax.set_title('Seaborn Confusion Matrix with labels\n\n')
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values')
## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['ADL','Fall'])
ax.yaxis.set_ticklabels(['ADL','Fall'])
## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
cnn_model_y = tf.keras.Sequential([
    Conv1D(filters = 64, kernel_size = 3, activation = 'relu'),
    Conv1D(filters = 64, kernel_size = 3, activation = 'relu'),
    Dropout(0.5),
    MaxPooling1D(pool_size = 2),
    Flatten(),
    layers.Dense(400, activation = 'relu'),
    layers.Dense(300, activation = 'relu'),
    layers.Dense(150, activation = 'relu'),
    layers.Dense(2, activation = 'sigmoid')
])

cnn_model_y.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
y_train_data_reshaped = y_train_data[..., None]
y_test_data_reshaped = y_test_data[..., None]

In [ ]:
history_y = cnn_model_y.fit(y_train_data_reshaped, y_train_labels,
                    validation_data = (y_test_data_reshaped, y_test_labels),
                    epochs = 100,
                    callbacks = [early_stopping],
                    shuffle = True)

In [ ]:
predicted_y = cnn_model_y.predict(y_test_data)
predicted_y = [np.argmax(i) for i in predicted_y]
actual_y = [np.argmax(i) for i in y_test_labels]

cf_matrix = confusion_matrix(predicted_y, actual_y)

ax = sn.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')
ax.set_title('Seaborn Confusion Matrix with labels\n\n')
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values')
## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['ADL','Fall'])
ax.yaxis.set_ticklabels(['ADL','Fall'])
## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
cnn_model_z = tf.keras.Sequential([
    Conv1D(filters = 64, kernel_size = 3, activation = 'relu'),
    Conv1D(filters = 64, kernel_size = 3, activation = 'relu'),
    Dropout(0.5),
    MaxPooling1D(pool_size = 2),
    Flatten(),
    layers.Dense(400, activation = 'relu'),
    layers.Dense(300, activation = 'relu'),
    layers.Dense(150, activation = 'relu'),
    layers.Dense(2, activation = 'sigmoid')
])

cnn_model_z.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
z_train_data_reshaped = z_train_data[..., None]
z_test_data_reshaped = z_test_data[..., None]

In [ ]:
history_z = cnn_model_z.fit(z_train_data_reshaped, z_train_labels,
                    validation_data = (z_test_data_reshaped, z_test_labels),
                    epochs = 100,
                    callbacks = [early_stopping],
                    shuffle = True)

In [ ]:
predicted_z = cnn_model_z.predict(z_test_data)
predicted_z = [np.argmax(i) for i in predicted_z]
actual_z = [np.argmax(i) for i in z_test_labels]

cf_matrix = confusion_matrix(predicted_z, actual_z)

ax = sn.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')
ax.set_title('Seaborn Confusion Matrix with labels\n\n')
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values')
## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['ADL','Fall'])
ax.yaxis.set_ticklabels(['ADL','Fall'])
## Display the visualization of the Confusion Matrix.
plt.show()

### 1D-CNN with Maximum Voting

In [ ]:
to_remove = sorted(np.unique(np.concatenate(([304,305,306],
[299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315],
[3,  54,  78, 141, 142, 152, 159, 170, 171, 172, 176, 207, 208, 209, 266, 267, 268, 297,
 299, 300, 301, 303, 304, 305, 306, 307, 309, 310, 313, 314, 315, 376, 399, 413, 428, 439,
 442, 514,]))))
to_keep = np.delete(np.arange(0,len(x_axis)), to_remove)

In [ ]:
x_adl = x_axis[to_keep]
y_adl = y_axis[to_keep]
z_adl = z_axis[to_keep]
x_data = np.concatenate([x_adl, x_fall])
x_data = np.concatenate([x_adl, x_fall])
x_data = np.concatenate([x_adl, x_fall])

x_labels = np.concatenate([np.zeros((len(x_adl),)), np.ones((len(x_fall),))])
y_labels = np.concatenate([np.zeros((len(x_adl),)), np.ones((len(x_fall),))])
z_labels = np.concatenate([np.zeros((len(x_adl),)), np.ones((len(x_fall),))])

x_labels = x_labels.reshape(-1, 1)
y_labels = y_labels.reshape(-1, 1)
z_labels = z_labels.reshape(-1, 1)

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(x_labels)

x_labels = enc.transform(x_labels).toarray()
y_labels = enc.transform(y_labels).toarray()
z_labels = enc.transform(z_labels).toarray()

In [ ]:
test_percent = 0.2
np.random.seed(100)
test_ind =  sorted(np.random.choice(range(x_data.shape[0]), replace = False, size = round(x_data.shape[0] * test_percent)))
train_ind = np.delete(np.arange(0, x_data.shape[0]), test_ind)

In [ ]:
x_train_data = x_data[train_ind, :]
y_train_data = y_data[train_ind, :]
z_train_data = z_data[train_ind, :]

x_train_labels = x_labels[train_ind, :]
y_train_labels = y_labels[train_ind, :]
z_train_labels = z_labels[train_ind, :]

x_test_data = x_data[test_ind, :]
y_test_data = y_data[test_ind, :]
z_test_data = z_data[test_ind, :]

x_test_labels = x_labels[test_ind, :]
y_test_labels = y_labels[test_ind, :]
z_test_labels = z_labels[test_ind, :]

In [ ]:
cnn_model_x = tf.keras.Sequential([
    Conv1D(filters = 64, kernel_size = 3, activation = 'relu'),
    Conv1D(filters = 64, kernel_size = 3, activation = 'relu'),
    Dropout(0.5),
    MaxPooling1D(pool_size = 2),
    Flatten(),
    layers.Dense(400, activation = 'relu'),
    layers.Dense(300, activation = 'relu'),
    layers.Dense(150, activation = 'relu'),
    layers.Dense(2, activation = 'sigmoid')
])

cnn_model_x.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
x_train_data_reshaped = x_train_data[..., None]
x_test_data_reshaped = x_test_data[..., None]

In [ ]:
history_x = cnn_model_x.fit(x_train_data_reshaped, x_train_labels,
                    validation_data = (x_test_data_reshaped, x_test_labels),
                    epochs = 100,
                    callbacks = [early_stopping],
                    shuffle = True)

In [ ]:
predicted_x = cnn_model_x.predict(x_test_data)
predicted_x = [np.argmax(i) for i in predicted_x]
actual_x = [np.argmax(i) for i in x_test_labels]

cf_matrix = confusion_matrix(predicted_x, actual_x)

ax = sn.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')
ax.set_title('Seaborn Confusion Matrix with labels\n\n')
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values')
## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['ADL','Fall'])
ax.yaxis.set_ticklabels(['ADL','Fall'])
## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
cnn_model_y = tf.keras.Sequential([
    Conv1D(filters = 64, kernel_size = 3, activation = 'relu'),
    Conv1D(filters = 64, kernel_size = 3, activation = 'relu'),
    Dropout(0.5),
    MaxPooling1D(pool_size = 2),
    Flatten(),
    layers.Dense(400, activation = 'relu'),
    layers.Dense(300, activation = 'relu'),
    layers.Dense(150, activation = 'relu'),
    layers.Dense(2, activation = 'sigmoid')
])

cnn_model_y.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
y_train_data_reshaped = y_train_data[..., None]
y_test_data_reshaped = y_test_data[..., None]

In [ ]:
history_y = cnn_model_y.fit(y_train_data_reshaped, y_train_labels,
                    validation_data = (y_test_data_reshaped, y_test_labels),
                    epochs = 100,
                    callbacks = [early_stopping],
                    shuffle = True)

In [ ]:
predicted_y = cnn_model_y.predict(y_test_data)
predicted_y = [np.argmax(i) for i in predicted_y]
actual_y = [np.argmax(i) for i in y_test_labels]

cf_matrix = confusion_matrix(predicted_y, actual_y)

ax = sn.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')
ax.set_title('Seaborn Confusion Matrix with labels\n\n')
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values')
## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['ADL','Fall'])
ax.yaxis.set_ticklabels(['ADL','Fall'])
## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
cnn_model_z = tf.keras.Sequential([
    Conv1D(filters = 64, kernel_size = 3, activation = 'relu'),
    Conv1D(filters = 64, kernel_size = 3, activation = 'relu'),
    Dropout(0.5),
    MaxPooling1D(pool_size = 2),
    Flatten(),
    layers.Dense(400, activation = 'relu'),
    layers.Dense(300, activation = 'relu'),
    layers.Dense(150, activation = 'relu'),
    layers.Dense(2, activation = 'sigmoid')
])

cnn_model_z.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
z_train_data_reshaped = z_train_data[..., None]
z_test_data_reshaped = z_test_data[..., None]

In [ ]:
history_z = cnn_model_z.fit(z_train_data_reshaped, z_train_labels,
                    validation_data = (z_test_data_reshaped, z_test_labels),
                    epochs = 100,
                    callbacks = [early_stopping],
                    shuffle = True)

In [ ]:
predicted_z = cnn_model_z.predict(z_test_data)
predicted_z = [np.argmax(i) for i in predicted_z]
actual_z = [np.argmax(i) for i in z_test_labels]

cf_matrix = confusion_matrix(predicted_z, actual_z)

ax = sn.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')
ax.set_title('Seaborn Confusion Matrix with labels\n\n')
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values')
## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['ADL','Fall'])
ax.yaxis.set_ticklabels(['ADL','Fall'])
## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
# Getting maximum votes
predicted_z = cnn_model_z.predict(z_test_data)
predicted_z = [np.argmax(i) for i in predicted_z]
actual_z = [np.argmax(i) for i in z_test_labels]

cf_matrix = confusion_matrix(predicted_z, actual_z)

ax = sn.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')
ax.set_title('Seaborn Confusion Matrix with labels\n\n')
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values')
## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['ADL','Fall'])
ax.yaxis.set_ticklabels(['ADL','Fall'])
## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
predicted = (np.array(predicted_x) + np.array(predicted_y) + np.array(predicted_z)) >= 2
actual = [np.argmax(i) for i in x_test_labels]

In [ ]:
# Getting maximum votes
cf_matrix = confusion_matrix(predicted, actual)

ax = sn.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')
ax.set_title('Seaborn Confusion Matrix with labels\n\n')
ax.set_xlabel('\nActual Values')
ax.set_ylabel('Predicted Values')
## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['ADL','Fall'])
ax.yaxis.set_ticklabels(['ADL','Fall'])
## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
count = 0
for i in range(len(predicted)):
    if predicted[i] == 1 and actual[i] == 0:
        count += 1
count

In [ ]:
# cnn_model_x.save("cnn_model_x_axis_UMAFall")
# cnn_model_y.save("cnn_model_y_axis_UMAFall")
# cnn_model_z.save("cnn_model_z_axis_UMAFall")
